# Advanced usage

This notebook shows some more advanced features of `skorch`. More examples will be added with time.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0);

### Table of contents

* [Custom callbacks](#Writing-custom-callbacks)

## Setup

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)

In [5]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### Definition of the `pytorch` classification `module`

We define a vanilla neural network with two hidden layers. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [6]:
from skorch.net import NeuralNetClassifier

In [7]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X

## Writing custom callbacks

Writing your own callbacks is also straightforward. Just remember these rules:
* They should inherit from `skorch.callbacks.Callback`.
* They should implement at least one of the `on_`-methods provided by the parent class (e.g. `on_batch_begin` or `on_epoch_end`).
* As argument, the `on_`-methods first get the `NeuralNet` instance, and, where appropriate, the local data (e.g. the data from the current batch). The method should also have `**kwargs` in the signature for potentially unused arguments.

Here is an example of a callback that saves the model if the validation loss has improved.

In [8]:
from skorch.callbacks import Callback


class Checkpoint(Callback):
    def __init__(self, file_name):
        self.file_name = file_name

    def on_epoch_end(self, net, **kwargs):
        # check if valid accuracy of most recent epoch is the best so far
        if net.history[-1, 'valid_acc_best']:
            print("Save model to {}.".format(self.file_name))
            net.save_params(self.file_name)

In [9]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    callbacks=[Checkpoint(file_name='/tmp/mymodel.pkl')],
    lr=0.1,
)

In [10]:
net.fit(X, y)

Save model to /tmp/mymodel.pkl.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7111       0.5100        0.6894  0.1296
Save model to /tmp/mymodel.pkl.
      2        0.6928       0.5500        0.6803  0.0597
Save model to /tmp/mymodel.pkl.
      3        0.6833       0.5650        0.6741  0.0541
Save model to /tmp/mymodel.pkl.
      4        0.6763       0.5850        0.6674  0.0536
Save model to /tmp/mymodel.pkl.
      5        0.6727       0.6450        0.6616  0.0544
Save model to /tmp/mymodel.pkl.
      6        0.6606       0.6600        0.6536  0.0455
      7        0.6560       0.6600        0.6443  0.0526
Save model to /tmp/mymodel.pkl.
      8        0.6427       0.6650        0.6354  0.0534
Save model to /tmp/mymodel.pkl.
      9        0.6300       0.6800        0.6264  0.0943
     10        0.6289       0.6800        0.6189  0.1214
Save model to /tmp/mymodel.pkl.
     11        0.6241       